In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install category_encoders
# !pip install dacon_submit_api-0.0.4-py3-none-any.whl
# !pip install mljar-supervised

In [3]:
## import 
import pandas as pd
import numpy as np
from supervised.automl import AutoML

# vis
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
%matplotlib inline
# matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치. 코랩에선 주석하셈
import seaborn as sns

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor 
from sklearn.ensemble import StackingClassifier, StackingRegressor
from sklearn.base import ClassifierMixin
# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# CatBoost
from catboost import CatBoostRegressor

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
# import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# 시드값 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2023)

# 데이터프레임 컬럼 전체 보기
pd.set_option('display.max_columns', None)

In [4]:
# 데이터 로드

path = '../data/daegu/automl/'

train_org = pd.read_csv(path+'train.csv')
test_org = pd.read_csv(path+'test.csv')

In [5]:
# 이희원님 코드 그대로 사용
for df in [train_org, test_org]:
    # 날짜 데이터
    time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

    df[['연', '월', '일', '시간']] = df['사고일시'].str.extract(time_pattern)
    df[['연', '월', '일', '시간']] = df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    # df= df.drop(columns=['사고일시']) # 일단 냄겨두자

    # 도시/구/동 분리
    location_pattern = r'(\S+) (\S+) (\S+)'

    df[['도시', '구', '동']] = df['시군구'].str.extract(location_pattern)
    # df= df.drop(columns=['시군구']) # 얘도 일단 냄겨놔

    # 도로형태도 분류 코드
    #road_pattern = r'(.+) - (.+)'

    #df[['도로형태1', '도로형태2']] = df['도로형태'].str.extract(road_pattern)
    #df= df.drop(columns=['도로형태']) # 얘는 드롭함

    # 시간 >> 코사인 변환
#     df['cos_hour'] = np.cos(2 * np.pi * df['시간'] / 24)
    # 📢송규헌) 사인-코사인 변환
    df['cos_hour'] = np.sign(np.pi*2*df['시간']/24) + np.cos(np.pi*2*df['시간']/24) + np.sign(np.pi*4*df['시간']/24) + np.cos(np.pi*4*df['시간']/24)

    # 월 >> 코사인 변환
    df['cos_month'] = -np.cos(np.pi*2*df['월']/12) + np.cos(np.pi*2*df['월']/12) - np.cos(np.pi*4*df['월']/12) + np.sign(np.pi*5*df['월']/12) + np.cos(np.pi*1*df['월']/12) + 2

    
    # 공휴일 (주말과 공휴일을 묶음)
    # 공휴일 = 1, 비공휴일 = 0
    holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']
    df['dow'] = df['사고일시'].dt.dayofweek #임시로 만든 컬럼
    df['holiday'] = np.where((df['dow']>=5) | (df['사고일시'].dt.strftime('%Y-%m-%d').isin(holi_weekday)),1,0)
    df.drop('dow',axis=1,inplace=True)
    
    # 계절 >> cos_month 사용할 때는 season은 생략
#     def group_season(df):
#         df.loc[(df['월'] == 3) | (df['월'] == 4) | (df['월'] == 5), 'season'] = '봄'
#         df.loc[(df['월'] == 6) | (df['월'] == 7) | (df['월'] == 8), 'season'] = '여름'
#         df.loc[(df['월'] == 9) | (df['월'] == 10) | (df['월'] == 11), 'season'] = '가을'
#         df.loc[(df['월'] == 12) | (df['월'] == 1) | (df['월'] == 2), 'season'] = '겨울'
#         return df['season']
#     df['season'] = group_season(df)

In [6]:
train_df = train_org
test_df = test_org

In [7]:
# load data
light_df = pd.read_csv(path+'동별_보안등개수.csv').rename(columns={'설치개수':'보안등개수'})
parking_df = pd.read_csv(path+'주차장급지구분별개수.csv')
speed_df = pd.read_csv(path+'동별_제한속도.csv')
cross_walk_df = pd.read_csv(path+'동별_횡단보도개수.csv')
child_area_df = pd.read_csv(path+'동별_어린이보호구역개수.csv')


# merge data
train_df = pd.merge(train_df, light_df, how='left', on=['구','동'])
train_df = pd.merge(train_df, parking_df, how='left', on=['구','동'])
train_df = pd.merge(train_df, speed_df, how='left', on='동')
train_df = pd.merge(train_df, cross_walk_df, how='left', on='동')
train_df = pd.merge(train_df, child_area_df, how='left', on=['구','동'])

test_df = pd.merge(test_df, light_df, how='left', on=['구','동'])
test_df = pd.merge(test_df, parking_df, how='left', on=['구','동'])
test_df = pd.merge(test_df, speed_df, how='left', on='동')
test_df = pd.merge(test_df, cross_walk_df, how='left', on='동')
test_df = pd.merge(test_df, child_area_df, how='left', on=['구','동'])


In [8]:
### 피처 추가...

# 시군구별 사고횟수 >> 기본 코드 사용
accident_counts = train_df['시군구'].value_counts().reset_index()
accident_counts.columns = ['시군구', '사고횟수']
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
train_df['사고발생횟수'] = train_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])
test_df['사고발생횟수'] = test_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])




In [9]:
# 📢송규헌) 연도값을 연도별 eclo평균값으로 변경
# 선형추세
yearly_eclo_avg = train_df.groupby('연')['ECLO'].mean()
x = np.array(yearly_eclo_avg.index)
y = yearly_eclo_avg.values
lr = LinearRegression()
lr.fit(x.reshape(-1,1),y.reshape(-1,1))
pred = lr.predict(np.array([2022]).reshape(-1,1))
yearly_eclo_avg[2022] = pred.reshape(1)[0]

train_df['연'] =  train_df['연'].map(yearly_eclo_avg)
test_df['연'] =  test_df['연'].map(yearly_eclo_avg)

In [10]:
# 점검
display(train_df.head(2))
display(test_df.head(2))

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,연,월,일,시간,도시,구,동,cos_hour,cos_month,holiday,보안등개수,급지구분_1,급지구분_2,급지구분_3,제한속도,횡단보도개수,어린이보호구역개수,사고발생횟수
0,ACCIDENT_00000,2019-01-01,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,4.842185,1,1,0,대구광역시,중구,대신동,2.0,3.465926,1,391.0,11.0,0.0,0.0,40.0,5929.0,2.0,131
1,ACCIDENT_00001,2019-01-01,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3,4.842185,1,1,0,대구광역시,달서구,감삼동,2.0,3.465926,1,941.0,0.0,1.0,3.0,52.5,10000.0,NaN,678


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,연,월,일,시간,도시,구,동,cos_hour,cos_month,holiday,보안등개수,급지구분_1,급지구분_2,급지구분_3,제한속도,횡단보도개수,어린이보호구역개수,사고발생횟수
0,ACCIDENT_39609,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,4.474125,1,1,1,대구광역시,수성구,상동,3.831951,3.465926,1,700.0,NaN,NaN,NaN,52.500000,9025.0,5.0,338
1,ACCIDENT_39610,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,4.474125,1,1,1,대구광역시,수성구,지산동,3.831951,3.465926,1,NaN,0.0,0.0,2.0,47.142857,52441.0,10.0,492


In [11]:
# 불필요 컬럼 제거
drop_columns = ['ID','도시','시간','시군구','사고일시','사고유형 - 세부분류','법규위반','가해운전자 차종','가해운전자 성별','가해운전자 연령',
       '가해운전자 상해정도','사망자수','중상자수','피해운전자 차종','피해운전자 성별',
      '피해운전자 연령','피해운전자 상해정도','경상자수','부상자수',
       '월'
]
train_df.drop(columns = drop_columns,inplace=True)
test_df.drop(columns=['ID','도시','시간','사고일시','시군구','월'
                     
                     
                     
                     ],inplace=True)

### 이상치 처리 및 결측값 처리

In [12]:
# 결측값 확인 
train_df.isnull().sum().sort_values(ascending=False)[:11]

어린이보호구역개수    18426
보안등개수        13813
급지구분_3        6543
급지구분_2        6543
급지구분_1        6543
제한속도          2442
횡단보도개수          62
요일               0
cos_month        0
holiday          0
cos_hour         0
dtype: int64

In [13]:
test_df.isnull().sum().sort_values(ascending=False)[:11]

어린이보호구역개수    4961
보안등개수        4029
급지구분_3       1928
급지구분_2       1928
급지구분_1       1928
제한속도          729
횡단보도개수         21
요일              0
기상상태            0
holiday         0
cos_month       0
dtype: int64

In [14]:
# # 일단 interpolate
# for df in [train_df,test_df]:
#     df['어린이보호구역개수'] = df['어린이보호구역개수'].interpolate()
#     df['급지구분_1'] = df['급지구분_1'].interpolate()
#     df['급지구분_2'] = df['급지구분_2'].interpolate()
#     df['급지구분_3'] = df['급지구분_3'].interpolate()
#     df['제한속도'] = df['제한속도'].interpolate()
#     df['횡단보도개수'] = df['횡단보도개수'].interpolate()
    
# # 급지구분 interpolate하면 안 채워지는 게 1개 있어서 0으로 대체
# test_df['급지구분_1'] = test_df['급지구분_1'].fillna(0)
# test_df['급지구분_2'] = test_df['급지구분_2'].fillna(0)
# test_df['급지구분_3'] = test_df['급지구분_3'].fillna(0)

In [15]:
# 결측값을 '구' 별 평균으로 처리
na_columns = ['어린이보호구역개수','급지구분_1','급지구분_1','급지구분_1'
             ,'제한속도','횡단보도개수','보안등개수']


### 어린이보호구역개수 결측값 처리
child_area_mean = train_df.groupby('구')['어린이보호구역개수'].mean()
# 달서구와 동구가 아예 어린이보호구역개수가 nan임.
child_mean_sum = child_area_mean.sum()/len(child_area_mean)
# 전체평균으로 대체 # 동구와 달성군 모두 면적이 넓음
child_area_mean['달서구'] = child_mean_sum
child_area_mean['동구'] = child_mean_sum
def fillna_with_mean_child(row):
    if np.isnan(row['어린이보호구역개수']):
        row['어린이보호구역개수'] = child_area_mean[row['구']]
    return row
train_df = train_df.apply(fillna_with_mean_child,axis=1)
test_df = test_df.apply(fillna_with_mean_child,axis=1)


### 급지구분 결측값 처리
parking_mean = train_df.groupby('구')[['급지구분_1','급지구분_2',
                          '급지구분_3']].mean()
cols = ['급지구분_1','급지구분_2','급지구분_3']
for col in cols:
    def fillna_with_mean_parking(row):
        if np.isnan(row[col]):
            row[col] = parking_mean.loc[row['구'],col]
        return row
    train_df = train_df.apply(fillna_with_mean_parking,axis=1)
    test_df = test_df.apply(fillna_with_mean_parking,axis=1)

    
### 보안등개수 결측값 처리
light_mean = train_df.groupby('구')['보안등개수'].mean()
# 달성군과 서구가 NaN >> 달성군은 면적이 크므로 전체평균값을 활용, 
# 서구는 작으므로 Q1 사용
light_total_mean = light_mean.sum()/len(light_mean)
light_mean['달성군'] = light_total_mean
light_mean['서구'] = light_total_mean * 0.25
def fillna_with_mean_light(row):
    if np.isnan(row['보안등개수']):
        row['보안등개수'] = light_mean[row['구']]
    return row
train_df = train_df.apply(fillna_with_mean_light,axis=1)
test_df = test_df.apply(fillna_with_mean_light,axis=1)

 
### 제한속도 결측값 처리
speed_mean = train_df.groupby('구')['제한속도'].mean()
def fillna_with_mean_speed(row):
    if np.isnan(row['제한속도']):
        row['제한속도'] = speed_mean[row['구']]
    return row
train_df = train_df.apply(fillna_with_mean_speed,axis=1)
test_df = test_df.apply(fillna_with_mean_speed,axis=1)


### 횡단보도개수 결측값 처리
cross_mean = train_df.groupby('구')['횡단보도개수'].mean()
def fillna_with_mean_cross(row):
    if np.isnan(row['횡단보도개수']):
        row['횡단보도개수'] = cross_mean[row['구']]
    return row
train_df = train_df.apply(fillna_with_mean_cross,axis=1)
test_df = test_df.apply(fillna_with_mean_cross,axis=1)

In [16]:
# 연습용 코드

# ### 제한속도 결측값 처리
# cross_mean = train_df.groupby('구')['횡단보도개수'].mean()
# def fillna_with_mean_speed(row):
#     if np.isnan(row['횡단보도개수']):
#         row['횡단보도개수'] = light_mean[row['구']]
#     return row
# train_df = train_df.apply(fillna_with_mean_speed,axis=1)
# test_df = test_df.apply(fillna_with_mean_speed,axis=1)



In [17]:
# 결측값 확인 
train_df.isnull().sum().sort_values(ascending=False)[:5]," ---- " , test_df.isnull().sum().sort_values(ascending=False)[:5]

(요일           0
 cos_month    0
 어린이보호구역개수    0
 횡단보도개수       0
 제한속도         0
 dtype: int64,
 ' ---- ',
 요일           0
 기상상태         0
 어린이보호구역개수    0
 횡단보도개수       0
 제한속도         0
 dtype: int64)

### 인코딩 및 스케일링

In [18]:
# 원핫 : 구

# train_df와 test_df의 기상상태 및 노면상태 열 선택
train_categorical_data = train_df[['구']]
test_categorical_data = test_df[['구']]

# OneHotEncoder 인스턴스 생성 및 fit_transform 수행
encoder = OneHotEncoder()
train_encoded = encoder.fit_transform(train_categorical_data)
test_encoded = encoder.transform(test_categorical_data)

# OneHotEncoder가 사용한 카테고리 목록을 가져와서 카테고리 이름을 열 이름으로 변환
feature_names = encoder.get_feature_names_out(['구'])

# 밀집 행렬로 변환 (선택 사항)
train_encoded_dense = train_encoded.toarray()
test_encoded_dense = test_encoded.toarray()

# 데이터프레임으로 변환 (선택 사항)
import pandas as pd

train_encoded_df = pd.DataFrame(train_encoded_dense, columns=feature_names, index=train_df.index)
test_encoded_df = pd.DataFrame(test_encoded_dense, columns=feature_names, index=test_df.index)

# 기존 열 제거
train_df = train_df.drop(['구'], axis=1)
test_df = test_df.drop(['구'], axis=1)

# 인코딩된 열 추가
train_df = pd.concat([train_df, train_encoded_df], axis=1)
test_df = pd.concat([test_df, test_encoded_df], axis=1)


In [19]:
# 타겟 인코딩 : 나머지
# Target encoding
from category_encoders.target_encoder import TargetEncoder

categorical_features = ['요일','도로형태','사고유형','동','기상상태', '노면상태']

for i in categorical_features:
    tr_encoder = TargetEncoder(cols=[i])
    train_df[i] = tr_encoder.fit_transform(train_df[i], train_df['ECLO'])
    test_df[i] = tr_encoder.transform(test_df[i])

### 기타 전처리

### 데이터셋 분할

In [20]:
train_x = train_df[test_df.columns] # test 셋에만 있는 컬럼만 추출
train_y = np.log1p(train_df['ECLO'])
test_x = test_df

In [21]:
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

### AUTOML

In [23]:
import matplotlib
matplotlib.use('Agg')

In [24]:
stop

NameError: name 'stop' is not defined

In [26]:
from supervised.automl import AutoML
automl = AutoML(mode="Compete",
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
                n_jobs = -1,total_time_limit=60000, eval_metric="rmse", ml_task = "regression",
               stack_models=True
               )

In [ ]:
automl.fit(train_x, train_y)

In [ ]:
pred = automl.predict(test_x)

## 제출

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')

In [ ]:
submission['ECLO'] = np.expm1(pred)

In [ ]:
submission.loc[ submission['ECLO'] < 0.0, 'ECLO'] = 0.0

In [ ]:
min(submission['ECLO'])

In [ ]:
submission

In [ ]:
# submission.to_csv('submit.csv', index=False)

submit_path = "../data/submits/automl_1207_002.csv"
submission.to_csv(submit_path, index=False)

In [ ]:
# 데이콘 api 제출

from dacon_submit_api import dacon_submit_api

result = dacon_submit_api.post_submission_file(
  submit_path,  # 제출 파일의 저장경로
  '99e503e9d7311db9f4167d982d5823d7ef9427cc628aa63b96cf79d2920ba916', # API 토큰
  '236193', # 대구공모전 ID
  'song999', # 맘대루
  '12.07 - 002. automl_some_updates' )  # 제출 메모